In [1]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import sklearn.ensemble
from sklearn.metrics import accuracy_score, roc_auc_score, balanced_accuracy_score, precision_score, matthews_corrcoef, confusion_matrix, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import statistics
import numpy as np

In [2]:
df = pd.read_csv("data/4.2_hg38_paired_introns.tsv", sep="\t")
df

,chr,start,end,strand,class,ss_antisense_start_site,ss_antisense_end_site,GTExv2,TCGAv2,SRAv3h,...,repeat_features_end_site:Dust,repeat_features_end_site:Unknown,repeat_features_end_site:RNA repeats,repeat_features_end_site:Satellite repeats,repeat_features_end_site:Tandem repeats,repeat_features_end_site:Low complexity regions,sequence,sequence_maxentscan,hg38 - Acceptor splice site,hg38 - Donor splice site
0,chr1,12227,12612,+,1,False,False,1122:3199,583:1460,9374:54492,...,0,0,0,0,0,0,GTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGCCGG...,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,GT,AG
1,chr1,12721,13220,+,1,False,False,1791:3198,783:1104,14048:56719,...,0,0,0,0,0,0,GTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTAGGG...,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,GT,AG
2,chr1,12057,12178,+,1,False,False,NaN,NaN,NaN,...,0,0,0,0,0,0,CACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGG...,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,CA,GA
3,chr1,12697,12974,+,1,False,False,1:1,1:1,15:19,...,0,0,0,0,0,0,GTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAGTGA...,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,GT,AG
4,chr1,13052,13220,+,1,False,False,22:24,17:18,433:484,...,0,0,0,0,0,0,GCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGGAGA...,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,GC,AG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527913,chrY,24883840,24886132,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,0,0,GTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGCCTT...,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...,GT,GC
527914,chrY,24888605,24889352,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,0,0,GTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAG...,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...,GT,AG
527915,chrY,24889386,24901111,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,0,0,ATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGACTTC...,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...,AT,AG
527916,chrY,24833970,24840730,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,0,0,CCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCACTCG...,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...,CC,AG


In [3]:
df_maxentscan_score = pd.read_csv("data/4.3_maxentscan", sep="\t")
df_maxentscan_score

,5_prime_site_maxent_seq,3_prime_site_maxent_seq,5_MaxEntScan,3_MaxEntScan
0,ccaGTAAGT,AGGCTCCTGTCTCCCCCCAGgtg,9.09,11.90
1,gagGTGAGA,TAGCCTCTGTTCCCACGAAGgca,7.66,3.89
2,gagCACTGG,AACCAGGCATAGGGGAAAGAttg,-13.87,-31.99
3,cttGTGAGT,GTCATCCCCTTCACTCCCAGctc,7.15,7.61
4,tagGCAAGC,TAGCCTCTGTTCCCACGAAGgca,1.14,3.89
...,...,...,...,...
527913,aatGTAAGA,CCATATTCATTTTTTTCTGCcaa,5.55,-9.69
527914,cagGTAATG,CTTTTTTTTTTTTTTTTAAGatg,9.43,10.01
527915,gggATTATG,TGAAAACTTCTGTGTTTCAGaaa,-14.46,7.85
527916,cgcCCTTGG,GTCTACTTTCCAAATTACAGtct,-27.63,4.64


In [4]:
df["5_MaxEntScan"] = df_maxentscan_score["5_MaxEntScan"]
df["3_MaxEntScan"] = df_maxentscan_score["3_MaxEntScan"]
df

,chr,start,end,strand,class,ss_antisense_start_site,ss_antisense_end_site,GTExv2,TCGAv2,SRAv3h,...,repeat_features_end_site:RNA repeats,repeat_features_end_site:Satellite repeats,repeat_features_end_site:Tandem repeats,repeat_features_end_site:Low complexity regions,sequence,sequence_maxentscan,hg38 - Acceptor splice site,hg38 - Donor splice site,5_MaxEntScan,3_MaxEntScan
0,chr1,12227,12612,+,1,False,False,1122:3199,583:1460,9374:54492,...,0,0,0,0,GTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGCCGG...,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,GT,AG,9.09,11.90
1,chr1,12721,13220,+,1,False,False,1791:3198,783:1104,14048:56719,...,0,0,0,0,GTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTAGGG...,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,GT,AG,7.66,3.89
2,chr1,12057,12178,+,1,False,False,NaN,NaN,NaN,...,0,0,0,0,CACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGG...,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,CA,GA,-13.87,-31.99
3,chr1,12697,12974,+,1,False,False,1:1,1:1,15:19,...,0,0,0,0,GTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAGTGA...,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,GT,AG,7.15,7.61
4,chr1,13052,13220,+,1,False,False,22:24,17:18,433:484,...,0,0,0,0,GCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGGAGA...,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,GC,AG,1.14,3.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527913,chrY,24883840,24886132,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,GTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGCCTT...,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...,GT,GC,5.55,-9.69
527914,chrY,24888605,24889352,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,GTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAG...,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...,GT,AG,9.43,10.01
527915,chrY,24889386,24901111,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,ATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGACTTC...,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...,AT,AG,-14.46,7.85
527916,chrY,24833970,24840730,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,CCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCACTCG...,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...,CC,AG,-27.63,4.64


In [5]:
del df_maxentscan_score

In [6]:
df = df.drop(columns=["GTExv2", "TCGAv2", "SRAv3h",], axis=1)
df

,chr,start,end,strand,class,ss_antisense_start_site,ss_antisense_end_site,RC3-Splice_site,RC3-Score,repeat_features_start_site:Centromere,...,repeat_features_end_site:RNA repeats,repeat_features_end_site:Satellite repeats,repeat_features_end_site:Tandem repeats,repeat_features_end_site:Low complexity regions,sequence,sequence_maxentscan,hg38 - Acceptor splice site,hg38 - Donor splice site,5_MaxEntScan,3_MaxEntScan
0,chr1,12227,12612,+,1,False,False,GT:AG,59151,0,...,0,0,0,0,GTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGCCGG...,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,GT,AG,9.09,11.90
1,chr1,12721,13220,+,1,False,False,GT:AG,61021,0,...,0,0,0,0,GTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTAGGG...,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,GT,AG,7.66,3.89
2,chr1,12057,12178,+,1,False,False,NaN,-1,0,...,0,0,0,0,CACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGG...,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,CA,GA,-13.87,-31.99
3,chr1,12697,12974,+,1,False,False,GT:AG,21,0,...,0,0,0,0,GTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAGTGA...,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,GT,AG,7.15,7.61
4,chr1,13052,13220,+,1,False,False,GC:AG,526,0,...,0,0,0,0,GCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGGAGA...,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,GC,AG,1.14,3.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527913,chrY,24883840,24886132,+,0,False,False,NaN,-1,0,...,0,0,0,0,GTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGCCTT...,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...,GT,GC,5.55,-9.69
527914,chrY,24888605,24889352,+,0,False,False,NaN,-1,0,...,0,0,0,0,GTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAG...,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...,GT,AG,9.43,10.01
527915,chrY,24889386,24901111,+,0,False,False,NaN,-1,0,...,0,0,0,0,ATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGACTTC...,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...,AT,AG,-14.46,7.85
527916,chrY,24833970,24840730,+,0,False,False,NaN,-1,0,...,0,0,0,0,CCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCACTCG...,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...,CC,AG,-27.63,4.64


In [7]:
df["splice-site"] = df["hg38 - Acceptor splice site"] + ":" + df["hg38 - Donor splice site"]


le = preprocessing.LabelEncoder()
le.fit(df["splice-site"])
df["splice-site"] = le.transform(df["splice-site"])
df

,chr,start,end,strand,class,ss_antisense_start_site,ss_antisense_end_site,RC3-Splice_site,RC3-Score,repeat_features_start_site:Centromere,...,repeat_features_end_site:Satellite repeats,repeat_features_end_site:Tandem repeats,repeat_features_end_site:Low complexity regions,sequence,sequence_maxentscan,hg38 - Acceptor splice site,hg38 - Donor splice site,5_MaxEntScan,3_MaxEntScan,splice-site
0,chr1,12227,12612,+,1,False,False,GT:AG,59151,0,...,0,0,0,GTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGCCGG...,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,GT,AG,9.09,11.90,182
1,chr1,12721,13220,+,1,False,False,GT:AG,61021,0,...,0,0,0,GTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTAGGG...,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,GT,AG,7.66,3.89,182
2,chr1,12057,12178,+,1,False,False,NaN,-1,0,...,0,0,0,CACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGG...,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,CA,GA,-13.87,-31.99,73
3,chr1,12697,12974,+,1,False,False,GT:AG,21,0,...,0,0,0,GTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAGTGA...,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,GT,AG,7.15,7.61,182
4,chr1,13052,13220,+,1,False,False,GC:AG,526,0,...,0,0,0,GCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGGAGA...,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,GC,AG,1.14,3.89,148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527913,chrY,24883840,24886132,+,0,False,False,NaN,-1,0,...,0,0,0,GTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGCCTT...,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...,GT,GC,5.55,-9.69,189
527914,chrY,24888605,24889352,+,0,False,False,NaN,-1,0,...,0,0,0,GTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAG...,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...,GT,AG,9.43,10.01,182
527915,chrY,24889386,24901111,+,0,False,False,NaN,-1,0,...,0,0,0,ATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGACTTC...,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...,AT,AG,-14.46,7.85,50
527916,chrY,24833970,24840730,+,0,False,False,NaN,-1,0,...,0,0,0,CCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCACTCG...,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...,CC,AG,-27.63,4.64,83


In [8]:
# Extra feature engineering
# TODO should be integrated back into the previous steps
df["intron_length"] = df["sequence"].apply(len)

In [9]:
# le = preprocessing.LabelEncoder()
# le.fit(df["hg38 - Acceptor splice site"])
# df["hg38 - Acceptor splice site"] = le.transform(df["hg38 - Acceptor splice site"])
#
# le = preprocessing.LabelEncoder()
# le.fit(df["hg38 - Donor splice site"])
# df["hg38 - Donor splice site"] = le.transform(df["hg38 - Donor splice site"])

le = preprocessing.LabelEncoder()
le.fit(df["splice-site"])
df["splice-site"] = le.transform(df["splice-site"])

le = preprocessing.LabelEncoder()
le.fit(df["ss_antisense_start_site"])
df["ss_antisense_start_site"] = le.transform(df["ss_antisense_start_site"])

le = preprocessing.LabelEncoder()
le.fit(df["ss_antisense_end_site"])
df["ss_antisense_end_site"] = le.transform(df["ss_antisense_end_site"])

df

,chr,start,end,strand,class,ss_antisense_start_site,ss_antisense_end_site,RC3-Splice_site,RC3-Score,repeat_features_start_site:Centromere,...,repeat_features_end_site:Tandem repeats,repeat_features_end_site:Low complexity regions,sequence,sequence_maxentscan,hg38 - Acceptor splice site,hg38 - Donor splice site,5_MaxEntScan,3_MaxEntScan,splice-site,intron_length
0,chr1,12227,12612,+,1,0,0,GT:AG,59151,0,...,0,0,GTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGCCGG...,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,GT,AG,9.09,11.90,182,385
1,chr1,12721,13220,+,1,0,0,GT:AG,61021,0,...,0,0,GTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTAGGG...,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,GT,AG,7.66,3.89,182,499
2,chr1,12057,12178,+,1,0,0,NaN,-1,0,...,0,0,CACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGG...,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,CA,GA,-13.87,-31.99,73,121
3,chr1,12697,12974,+,1,0,0,GT:AG,21,0,...,0,0,GTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAGTGA...,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,GT,AG,7.15,7.61,182,277
4,chr1,13052,13220,+,1,0,0,GC:AG,526,0,...,0,0,GCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGGAGA...,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,GC,AG,1.14,3.89,148,168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527913,chrY,24883840,24886132,+,0,0,0,NaN,-1,0,...,0,0,GTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGCCTT...,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...,GT,GC,5.55,-9.69,189,2292
527914,chrY,24888605,24889352,+,0,0,0,NaN,-1,0,...,0,0,GTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAG...,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...,GT,AG,9.43,10.01,182,747
527915,chrY,24889386,24901111,+,0,0,0,NaN,-1,0,...,0,0,ATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGACTTC...,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...,AT,AG,-14.46,7.85,50,11725
527916,chrY,24833970,24840730,+,0,0,0,NaN,-1,0,...,0,0,CCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCACTCG...,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...,CC,AG,-27.63,4.64,83,6760


In [10]:
df_annot_test_y = df.pop("class")
df_annot_test_y

0         1
1         1
2         1
3         1
4         1
         ..
527913    0
527914    0
527915    0
527916    0
527917    0
Name: class, Length: 527918, dtype: int64

In [11]:
df["RC3-Score-25"] = df["RC3-Score"] >= 25

# Drop the columns that might not be available prior to manual gene annotation or irrelevant to ML
df_annot_test_X_1 = df.drop(["chr", "start", "end", "strand", "RC3-Splice_site", "sequence", "hg38 - Donor splice site", "hg38 - Acceptor splice site", "sequence_maxentscan",
                             "repeat_features_end_site:Unknown", "repeat_features_start_site:Dust", "repeat_features_end_site:Dust", "repeat_features_start_site:RNA repeats",
                             "repeat_features_end_site:RNA repeats", "repeat_features_end_site:Centromere", "repeat_features_start_site:Centromere", "repeat_features_start_site:Unknown",
                             "repeat_features_start_site:Low complexity regions", "repeat_features_end_site:Low complexity regions"], axis=1)

# FIXME
# df_annot_test_X_1 = df_annot_test_X_1[(df_annot_test_X_1["hg38 - Acceptor splice site"] == 13) & (df_annot_test_X_1["hg38 - Donor splice site"] == 2)]

del df

In [12]:
df_annot_test_y

0         1
1         1
2         1
3         1
4         1
         ..
527913    0
527914    0
527915    0
527916    0
527917    0
Name: class, Length: 527918, dtype: int64

In [13]:
df_annot_test_X_1

,ss_antisense_start_site,ss_antisense_end_site,RC3-Score,repeat_features_start_site:Type II Transposons,repeat_features_start_site:Type I Transposons/SINE,repeat_features_start_site:Simple repeats,repeat_features_start_site:Type I Transposons/LINE,repeat_features_start_site:LTRs,repeat_features_start_site:Satellite repeats,repeat_features_start_site:Tandem repeats,...,repeat_features_end_site:Simple repeats,repeat_features_end_site:Type I Transposons/LINE,repeat_features_end_site:LTRs,repeat_features_end_site:Satellite repeats,repeat_features_end_site:Tandem repeats,5_MaxEntScan,3_MaxEntScan,splice-site,intron_length,RC3-Score-25
0,0,0,59151,0,0,0,0,0,0,0,...,0,0,0,0,0,9.09,11.90,182,385,True
1,0,0,61021,0,0,0,0,0,0,0,...,0,0,0,0,0,7.66,3.89,182,499,True
2,0,0,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,-13.87,-31.99,73,121,False
3,0,0,21,0,0,0,0,0,0,0,...,0,0,0,0,0,7.15,7.61,182,277,False
4,0,0,526,0,0,0,0,0,0,0,...,0,0,0,0,0,1.14,3.89,148,168,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527913,0,0,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,5.55,-9.69,189,2292,False
527914,0,0,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,9.43,10.01,182,747,False
527915,0,0,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,-14.46,7.85,50,11725,False
527916,0,0,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,-27.63,4.64,83,6760,False


In [14]:
X_train_val = df_annot_test_X_1.copy()
y_train_val = df_annot_test_y.copy()
del df_annot_test_X_1
del df_annot_test_y

In [15]:
cv_acc_list = []
cv_ba_acc_list = []
cv_rocauc_list = []
cv_precision_list = []
cv_recall_list = []
cv_mcc_list = []
cv_specificity_list = []
cv_sensitivity_list = []
cv_fpr_list = []
cv_fnr_list = []
model_dict = {}
model_index = 0
matrix = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
skf.get_n_splits(X_train_val, y_train_val)

for train_index, val_index in skf.split(X_train_val, y_train_val):
    X_train_resampled = X_train_val.iloc[train_index]
    y_train_resampled = y_train_val.iloc[train_index].to_numpy().flatten()

    X_train, X_val = X_train_resampled, X_train_val.iloc[val_index]
    y_train, y_val = y_train_resampled, y_train_val.iloc[val_index].to_numpy().flatten()

    model_dict[model_index] = XGBClassifier(seed=0)
    model_dict[model_index].fit(X_train, y_train)

    y_predict = model_dict[model_index].predict_proba(X_val)
    y_predict = y_predict[:, 1]
    # y_predict_class = list(map(round, y_predict))
    y_predict_class = [1 if pred >= 0.77 else 0 for pred in y_predict] #0.77

    test_acc = accuracy_score(y_val, y_predict_class)
    test_rocauc = roc_auc_score(y_val, y_predict)
    test_bal_acc = balanced_accuracy_score(y_val, y_predict_class)
    test_precision = precision_score(y_val, y_predict_class)  # tp/(tp+fp)
    test_recall = recall_score(y_val, y_predict_class)
    test_mcc = matthews_corrcoef(y_val, y_predict_class)
    matrix.append(confusion_matrix(y_val, y_predict_class))
    tn, fp, fn, tp = confusion_matrix(y_val, y_predict_class).ravel()

    # Adding the metrics to their list
    cv_acc_list.append(test_acc)
    cv_ba_acc_list.append(test_bal_acc)
    cv_rocauc_list.append(test_rocauc)
    cv_precision_list.append(test_precision)
    cv_mcc_list.append(test_mcc)
    cv_recall_list.append(test_recall)
    cv_specificity_list.append(tn / (tn + fp))
    cv_sensitivity_list.append(tp / (fn + tp))
    cv_fpr_list.append(fp/(fp+tn))
    cv_fnr_list.append(fn/(fn+tp))

In [ ]:
print("------------------------------------------------------------------------------------------")
print("Stratified Cross-Validation Performance (* metrics of interest)")
print("------------------------------------------------------------------------------------------")
print("*Accuracy: %s \nAUCROC: %s \n*MCC: %s \nSensitivity: %s \nSpecificity: %s \n*Precision: %s \nRecall: %s \n*False Positive Rate: %s \n*False Negative Rate: %s \nBalanced Accuracy: %s" % (
        statistics.mean(cv_acc_list), statistics.mean(cv_rocauc_list), statistics.mean(cv_mcc_list),
        statistics.mean(cv_sensitivity_list), statistics.mean(cv_specificity_list), statistics.mean(cv_precision_list), statistics.mean(cv_recall_list), statistics.mean(cv_fpr_list), statistics.mean(cv_fnr_list), statistics.mean(cv_ba_acc_list)))

print("------------------------------------------------------------------------------------------")
print("Accuracy SD: %s \nAUCROC SD: %s \nMCC SD: %s \nSensitivity SD: %s \nSpecificity SD: %s \nPrecision SD: %s  \nRecall SD: %s \n*False Positive Rate SD: %s \n*False Negative Rate SD: %s \nBalanced Accuracy SD: %s" % (
        np.std(cv_acc_list), np.std(cv_rocauc_list), np.std(cv_mcc_list),
        np.std(cv_sensitivity_list), np.std(cv_specificity_list), np.std(cv_precision_list), np.std(cv_recall_list), np.std(cv_fpr_list), np.std(cv_fnr_list), np.std(cv_ba_acc_list)))


print("------------------------------------------------------------------------------------------")

# Print the matrix
(sum(matrix))

------------------------------------------------------------------------------------------
Stratified Cross-Validation Performance (* metrics of interest)
------------------------------------------------------------------------------------------
*Accuracy: 0.9424304551988614 
AUCROC: 0.9800618741245941 
*MCC: 0.8504522807137903 
Sensitivity: 0.9425151643025904 
Specificity: 0.9421586954841602 
*Precision: 0.9812276031596654 
Recall: 0.9425151643025904 
*False Positive Rate: 0.057841304515839825 
*False Negative Rate: 0.05748483569740962 
Balanced Accuracy: 0.9423369298933753
------------------------------------------------------------------------------------------
Accuracy SD: 0.0010480960034714012 
AUCROC SD: 0.0005090650368659639 
MCC SD: 0.002534599618987599 
Sensitivity SD: 0.0011976941876172408 
Specificity SD: 0.0015606270726833595 
Precision SD: 0.0005018465412182332  
Recall SD: 0.0011976941876172408 
*False Positive Rate SD: 0.0015606270726833498 
*False Negative Rate SD: 0.00

array([[118207,   7257],
       [ 23135, 379319]])

# Previous model:

| Metric               | Value                    | Standard Deviation       |
|----------------------|--------------------------|--------------------------|
| Accuracy             | 0.7983640800390435       | 0.007369779726815882     |
| AUCROC               | 0.868038309019828        | 0.011973584999514585     |
| MCC                  | 0.040672051294380555     | 0.0018166924023484236    |
| Sensitivity          | 0.7983729490685108       | 0.0073908204378677214    |
| Specificity          | 0.7869620253164558       | 0.03261376499333584      |
| Precision            | 0.9997921983516019       | 3.094276128538986e-05    |
| Recall               | 0.7983729490685108       |                          |
| False Positive Rate  | 0.21303797468354432      |                          |
| Balanced Accuracy    | 0.7926674871924831       | 0.014386556944333703     |


<br/>

|                     | Predicted Negative | Predicted Positive |
|---------------------|--------------------|--------------------|
| Actual Negative     | 314                | 85                 |
| Actual Positive     | 103203             | 408648             |



|                     | Model 1 | Model 2 (with simulated reads)| Model 2 + Antisense feature + more false introns |
|---------------------|---------|-------------------------------|-------------------------------------------------|
| Accuracy            | 0.798   | 0.955                         | 0.923                                           |
| MCC                 | 0.040   | 0.745                         | 0.786                                           |
| Precision           | 0.999   | 0.967                         | 0.942                                           |
| False Positive Rate | 0.213   | 0.291                         | 0.188                                           |
<br/>

|                     | Predicted Negative | Predicted Positive |
|---------------------|--------------------|--------------------|
| Actual Negative     | 101786             | 23678              |
| Actual Positive     | 16487              | 385967              |

The increase in MCC and Accuracy is significant, we have to account for the class imbalance in model 1 and that guarentees to have high precision rate and false positive rate, as now we have much more negatively-lablled data, the decrease in precision and increase false positive rate is still relatively low.
Overall we can conclude that generating the false-set helps the model greatly.

In [17]:
# Next we will evaluate the feature importance of the XGBoost model.
def evaluate_feature_importance(df_evaluate_feature_importance, md):
    feature_importance_average = [np.mean(i) for i in zip(*[md.get(model).feature_importances_ for model in md])]
    df_feature_importance = pd.DataFrame()
    df_feature_importance["Property"] = df_evaluate_feature_importance.columns
    df_feature_importance["Average_importance"] = feature_importance_average
    print(df_feature_importance.sort_values(by="Average_importance", ascending=False).head(50))

evaluate_feature_importance(df_evaluate_feature_importance=X_train_val.copy(), md=model_dict)

                                             Property  Average_importance
2                                           RC3-Score            0.468801
14                      repeat_features_end_site:LTRs            0.062479
7                     repeat_features_start_site:LTRs            0.054129
6   repeat_features_start_site:Type I Transposons/...            0.051592
11   repeat_features_end_site:Type I Transposons/SINE            0.050494
13   repeat_features_end_site:Type I Transposons/LINE            0.043008
4   repeat_features_start_site:Type I Transposons/...            0.036819
18                                       3_MaxEntScan            0.033730
0                             ss_antisense_start_site            0.030446
3      repeat_features_start_site:Type II Transposons            0.029737
10       repeat_features_end_site:Type II Transposons            0.029730
17                                       5_MaxEntScan            0.028538
1                               ss_ant

 # Improvements

We can see that Centromere, RNA repeats region, Unknown, and Dust region has the lowest importance, which is consistent with our hypothesis, we should be removing them from the features.

In [18]:
X_train_val["class"] = y_train_val

In [19]:
X_train_val[(X_train_val["class"] == 0)]["intron_length"].median()

1911.0

In [20]:
df[(df["class"] == 1) & (df["splice-site"] != 182)]

NameError: name 'df' is not defined